In [19]:
# coding:utf-8
import requests
import hashlib
import urllib2
import urllib
import urlparse
from bs4 import BeautifulSoup

def crawl(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36'
    headers = {'User-Agent': user_agent}
    request = urllib2.Request(url, headers=headers)
    response = urllib2.urlopen(request) 
    html = response.read()
    soup = BeautifulSoup(html, from_encoding='utf-8')
    return soup

def parse(soup, is_homepage):
    results = soup.find('div', id='content_left').find_all('div', class_='result')
    ans = []
    for result in results:
        # title: 新闻标题
        # title_url: 新闻链接
        # source_and_time: 作者/时间
        # summary: 摘要
        # simi_words: 相同新闻
        # simi_words_url: 相同新闻查询的url
        title = result.find('h3', class_='c-title').find('a').get_text()
        title_url = result.find('h3', class_='c-title').find('a')['href']
        try:
            abstract = result.find('div', class_='c-summary c-row ').get_text()
        except:
            abstract = result.find('div', class_='c-span18 c-span-last').get_text()
        source_and_time = result.find('p', class_='c-author').get_text()
        temp_abstract = abstract[len(source_and_time):]
        summary = ""
        for word in abstract:
            summary = summary + word
            if summary[-3:] == '...':
                break
        temp_list = [title, title_url, source_and_time, summary]
        try:
            simi_words = results[0].find('a', class_='c-more_link').get_text()
            simi_words_url = 'http://news.baidu.com' + results[0].find('a', class_='c-more_link')['href']
            temp_list.append(simi_words)
            temp_list.append(simi_words_url)
        except:
            pass
        ans.append(temp_list)

    if is_homepage == 1:
        page_list = []
        try:    
            pages = soup.find('p', id='page').find_all('a')
            for page in pages:
                page_list.append('http://news.baidu.com' + page['href'])
        except:
            pass

        return page_list, ans
    else:
        return ans

def search(query_word):
    word = urllib.quote(query_word)
    url  = 'http://news.baidu.com/ns?cl=2&rn=40&tn=news&word='+word+'&pn=0'
    soup = crawl(url)
    page_list, ans = parse(soup, 1)
    return page_list, ans

def get_more(page_list):
    ans = []
    for page in page_list:
        soup = crawl(page)
        temp_ans = parse(soup, 0)
        ans = ans + temp_ans

    return ans

def get_same(url):
    soup = crawl(url)
    ans = parse(soup, 0)
    return ans

In [26]:
plist, ans = search('军民融合') #测试搜索词语

In [25]:
# 测试相同新闻
print ans[0][5]
same_news = get_same(ans[0][5])
print len(same_news)

http://news.baidu.com/ns?word=%E5%86%9B%E6%B0%91%E8%9E%8D%E5%90%88+cont:2987616210&same=4&cl=1&tn=news&rn=30&fm=sd
4


In [27]:
all_other_news = get_more(plist[1:])# 测试 more
print len(all_other_news)

360
